# スライド生成入門

このノートブックでは、LangChainとLangGraphを使用してPDFやHTMLドキュメントからスライドプレゼンテーションを生成する基本的な流れを紹介します。

## 概要

このプロセスは以下のステップで構成されています：

1. ドキュメントの読み込みと前処理
2. コンテンツ構造の抽出
3. スライドアウトラインの生成
4. 詳細なスライドコンテンツの生成
5. HTMLスライドの作成

それでは始めましょう！

## 環境のセットアップ

In [ ]:
# 必要なライブラリのインポート
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む
load_dotenv()

# プロジェクトルートへのパスを追加
root_dir = Path().absolute().parent
sys.path.append(str(root_dir))

# APIキーの確認
required_keys = ["OPENAI_API_KEY", "ANTHROPIC_API_KEY"]
missing_keys = [key for key in required_keys if os.getenv(key) is None]

if missing_keys:
    print(f"警告: 次のAPIキーが設定されていません: {', '.join(missing_keys)}")
    print("プロジェクトルートの.envファイルにAPIキーを設定してください")
else:
    print("APIキーが正常に設定されています")

## LangChain関連のインポート

In [ ]:
from langchain.document_loaders import PyPDFLoader, UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chat_models import ChatAnthropic, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel, Field
import langgraph.graph as lg
from typing import List, Dict, Any, Optional

# 使用するLLMの設定
model_name = os.getenv("DEFAULT_MODEL", "claude-3-haiku-20240307")

if model_name.startswith("claude"):
    llm = ChatAnthropic(model=model_name)
elif model_name.startswith("gpt"):
    llm = ChatOpenAI(model=model_name)
else:
    raise ValueError(f"サポートされていないモデル: {model_name}")

print(f"使用するモデル: {model_name}")

## ドキュメント処理のユーティリティ

In [ ]:
class DocumentProcessor:
    def __init__(self, chunk_size=1000, chunk_overlap=200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        
    def load_document(self, file_path: str) -> List[Document]:
        """ファイルタイプに応じたローダーを使用"""
        if file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif file_path.endswith('.html'):
            loader = UnstructuredHTMLLoader(file_path)
        else:
            raise ValueError(f"サポートされていないファイルタイプ: {file_path}")
        
        return loader.load()
    
    def process_documents(self, docs: List[Document]) -> List[Document]:
        """ドキュメントをチャンクに分割"""
        return self.text_splitter.split_documents(docs)
    
    def extract_toc(self, docs: List[Document]) -> List[str]:
        """目次を抽出する試み (簡易実装)"""
        # 実際のプロジェクトではもっと洗練されたロジックが必要
        potential_headings = []
        for doc in docs:
            lines = doc.page_content.split('\n')
            for line in lines:
                # 見出しらしい行を抽出 (単純な例)
                if line.strip() and len(line.strip()) < 100 and not line.endswith('.'):
                    potential_headings.append(line.strip())
        
        return potential_headings[:10]  # 単純化のため最初の10項目のみ

## スライド生成のためのモデル定義

In [ ]:
class SlideSection(BaseModel):
    title: str = Field(description="スライドのセクションタイトル")
    content: List[str] = Field(description="箇条書きのコンテンツリスト")
    notes: Optional[str] = Field(description="講演者ノート（オプショナル）", default=None)

class SlidePresentation(BaseModel):
    title: str = Field(description="プレゼンテーションのタイトル")
    subtitle: Optional[str] = Field(description="サブタイトル（オプショナル）", default=None)
    sections: List[SlideSection] = Field(description="プレゼンテーションのセクション")

## LangGraphワークフローのスケッチ

次のコードは実際に実行できる完全な実装ではなく、LangGraphを使ったワークフローの基本的な構造を示すスケッチです。詳細な実装は次のノートブックで行います。

In [ ]:
def create_slide_generation_workflow():
    # ノードの定義
    
    @lg.node
    def extract_content_structure(state):
        """ドキュメントから主要な構造と内容を抽出"""
        # 実装は次のノートブックで
        return {"content_structure": "構造の抽出結果", **state}
    
    @lg.node
    def generate_slide_outline(state):
        """抽出された構造からスライドアウトラインを生成"""
        # 実装は次のノートブックで
        return {"slide_outline": "アウトラインの生成結果", **state}
    
    @lg.node
    def generate_detailed_slides(state):
        """アウトラインから詳細なスライド内容を生成"""
        # 実装は次のノートブックで
        return {"slide_presentation": "詳細スライドの生成結果", **state}
    
    @lg.node
    def generate_html_slides(state):
        """構造化されたスライドデータからHTMLを生成"""
        # 実装は次のノートブックで
        return {"html_output": "HTML出力の生成結果", **state}
    
    # グラフの定義
    workflow = lg.Graph()
    workflow.add_node("extract_content_structure", extract_content_structure)
    workflow.add_node("generate_slide_outline", generate_slide_outline)
    workflow.add_node("generate_detailed_slides", generate_detailed_slides)
    workflow.add_node("generate_html_slides", generate_html_slides)
    
    # エッジの定義
    workflow.add_edge("extract_content_structure", "generate_slide_outline")
    workflow.add_edge("generate_slide_outline", "generate_detailed_slides")
    workflow.add_edge("generate_detailed_slides", "generate_html_slides")
    
    # コンパイル
    app = workflow.compile()
    
    return app

# 実際のワークフローの詳細な実装は次のノートブックで行います

## 次のステップ

このノートブックでは、スライド生成システムの基本的な構造を紹介しました。次のノートブックでは、以下のトピックを詳しく説明します：

1. より詳細なドキュメント処理と構造抽出
2. LLMを使用したコンテンツ分析とスライド生成
3. LangGraphを使用した完全なワークフローの実装
4. HTMLテンプレートの活用とスタイルのカスタマイズ

次のノートブック `01_document_processing.ipynb` に進んで、実際の実装を始めましょう！